<a href="https://colab.research.google.com/github/tolewicz/Product_Reviews_Big_Data/blob/main/Analysis_BIg_Data_challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install Java, Spark, and Findspark
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
!tar xf spark-2.4.5-bin-hadoop2.7.tgz
!pip install -q findspark
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

--2020-05-05 05:31:28--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.2’

postgresql-42.2.9.j 100%[===================>] 892.61K  3.85MB/s    in 0.2s    

2020-05-05 05:31:29 (3.85 MB/s) - ‘postgresql-42.2.9.jar.2’ saved [914037/914037]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [ ]:
# Read in data from S3 Buckets user_data.csv
#IT WORKED !!  
from pyspark import SparkFiles
url ="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_Games_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
user_data_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Video_Games_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)


# Show DataFrame
user_data_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   12039526| RTIS3L2M1F5SM|B001CXYMFS|     737716809|Thrustmaster T-Fl...|     Video Games|          5|            0|          0|   N|                Y|an amazing joysti...|Used this for Eli...|2015-08-31 00:00:00|
|         US|    9636577| R1ZV7R40OLHKD|B00M920ND6|     569686175|Tonsee 6 buttons ...| 

In [ ]:
print(f'number of rows = {user_data_df.count()}')
print(f'number of columns {len(user_data_df.columns)}')

number of rows = 1785997
number of columns 15


In [ ]:
#Dropping nan
data_clean_na=user_data_df.dropna()
print(f'number of rows data na cleanned = {data_clean_na.count()}')

#Dropping the same rows
data_clean_double = data_clean_na.distinct()
print(f'number of rows data wo 2x rows = {data_clean_double.count()}')

#Dropping the data of same customer reviewing same product 2x, 
#Cancelled because it did not add any bennefit based on the result and it slowed down the process
# data_clean=data_clean_double.dropDuplicates(["customer_id","product_id"])
# print(f'number of rows data wo 2x rows = {data_clean.count()}')

#format daytime to date
from pyspark.sql.types import DateType
data_clean=data_clean_double.withColumn("review_date",data_clean_double["review_date"].cast(DateType()))

analysis_table = data_clean.select(['star_rating','vine'])



number of rows data na cleanned = 1785886
number of rows data wo 2x rows = 1785886


In [ ]:
#verified purcahse
from pyspark.sql.functions import col

In [ ]:
#filtering for "with vine review"
vine_cleaned = analysis_table.filter(col("vine") == "Y")

#filtering for "without vine reviews"
no_vine_cleaned = analysis_table.filter(col("vine") == "N")

#gettign describe statistics for Vine/no Vine reviews
print('print with vine')
vine_cleaned.describe("star_rating").show()

print('no_vine')
no_vine_cleaned.describe("star_rating").show()

print with vine
+-------+------------------+
|summary|       star_rating|
+-------+------------------+
|  count|              4290|
|   mean|4.0748251748251745|
| stddev|0.9183222304948364|
|    min|                 1|
|    max|                 5|
+-------+------------------+

no_vine
+-------+-----------------+
|summary|      star_rating|
+-------+-----------------+
|  count|          1781596|
|   mean| 4.05985981109073|
| stddev|1.356685162601055|
|    min|                1|
|    max|                5|
+-------+-----------------+



In [ ]:
#Corelation of vine review with purchase , and getting the product rating
#Partitioning out the customer impact
#data purchsed has review based on personal experience
#data not purchased is only opininon
vine_data_purchased = vine_cleaned.filter(col("verified_purchase") == "Y") 
vine_data_not_purcahsed = vine_cleaned.filter(col("verified_purchase") == "N")
no_vine_data_purcahsed = no_vine_cleaned.filter(col("verified_purchase") == "Y")
no_vine_data_not_purcahsed = no_vine_cleaned.filter(col("verified_purchase") == "N")

print("Purchased product with vine")
vine_data_purchased.describe("star_rating").show()

print("Purchased product without vine")
no_vine_data_purcahsed.describe("star_rating").show()

print("Not purchased product with vine")
vine_data_not_purcahsed.describe("star_rating").show()

print("Not purchased product without vine - e.g. some one didn't want to buy and wrote bad opinion")
no_vine_data_not_purcahsed.describe("star_rating").show()

Purchased product with vine
+-------+------------------+
|summary|       star_rating|
+-------+------------------+
|  count|                22|
|   mean| 4.454545454545454|
| stddev|0.9625003513801574|
|    min|                 1|
|    max|                 5|
+-------+------------------+

Purchased product without vine
+-------+------------------+
|summary|       star_rating|
+-------+------------------+
|  count|           1169594|
|   mean| 4.191541680275377|
| stddev|1.2794382872741497|
|    min|                 1|
|    max|                 5|
+-------+------------------+

Not purchased product with vine
+-------+------------------+
|summary|       star_rating|
+-------+------------------+
|  count|              4268|
|   mean| 4.072867853795689|
| stddev|0.9178001865545383|
|    min|                 1|
|    max|                 5|
+-------+------------------+

Not purchased product without vine - e.g. some one didn't want to buy and wrote bad opinion
+-------+------------------+
|s

In the first glance bias of Vine on the product review seem to be negligible:

Rating with vine is 4.07
Rating w/o vine is 4.05

Howerer, once the data is filtered by purchased product and compared vine vs no vine the difference is larger: with vine 4.45, without vine 4.19

The reason for such filtering is that, the opinion about purchased product comes from the user, not from a bot.

To verify if the vine ratings is biased I looked at the review of product purchased and not purchsed with vine. It turned out that the number of purchased products with vine is only 22, which means that most of the vine opinions (4268) was done without purchasing the product, which strngly suggests that the vine opinion is biased